Report Extraction

In [0]:
import pandas as pd
import numpy as np

from datetime import date
from datetime import datetime
from datetime import timedelta


In [0]:
Today = date.today()

check_date = date.today() - timedelta(days = 90)

print('Today is '+ str(Today), ', check_date is '+ str(check_date))

Today is 2022-08-25 , check_date is 2022-05-27

In [0]:
quarter_start = pd.to_datetime(check_date - pd.tseries.offsets.QuarterBegin(startingMonth=1)).date()
quarter_end = pd.to_datetime(check_date + pd.tseries.offsets.QuarterEnd(startingMonth=3)).date()

print(quarter_start,quarter_end)

2022-04-01 2022-06-30

In [0]:
print("The checking quarter start from '{}' and end at '{}'".format(quarter_start,quarter_end))


The checking quarter start from '2022-04-01' and end at '2022-06-30'

In [0]:
query_result = spark.sql("""
select distinct
chdrcoy as company_code,
b.proposal_num as aia_policy_no,
chdrnum as IL_contract_no,
letter_type,
hsublet as sub_letter_type,
zsldrsn as reason_cd,
datefrm as suppression_date_from,
dateto as suppression_date_to,
validflag,
user_profile,
date_format(datime, 'yyyy-MM-dd') as date_of_suppression,
zappusr as Endorser_lanid
from ops_aos_read.il_zsyppf a 
left join ods_hist.adam_policy_hist b 
on a.chdrnum = b.pol_num
where ods_valid_ind = 1 and validflag = '1' and date_trunc('DD',a.datime) >= '{}' and date_trunc('DD',a.datime) <= '{}' 
""".format(quarter_start,quarter_end))


In [0]:
Query_result_df = query_result.toPandas()
Query_result_df.head()

Out[7]:

,company_code,aia_policy_no,IL_contract_no,letter_type,sub_letter_type,reason_cd,suppression_date_from,suppression_date_to,validflag,user_profile,date_of_suppression,Endorser_lanid
0,2,E241882074,69133899,ZPOSCOA,ZPOSCOA,SYSR,20220408,20220409,1,SSSO333,2022-04-12,SSSO333
1,2,H241176528,65587028,ZPOSCOA,ZPOSCOA,SYSR,20220402,20220403,1,SSSO333,2022-04-04,SSSO333
2,2,H235426899,04219516,ZCMCTMSR,ZCMCTMSR,SYSR,20220603,20220604,1,SSRV326,2022-06-06,SSRV326
3,2,H239809069,57590545,ZPOSCOA,ZPOSCOA,SYSR,20220411,20220412,1,SSSO333,2022-04-12,SSSO333
4,2,H241033056,64771321,ZPOSCOA,ZPOSCIP,SYSR,20220528,20220529,1,SSSO333,2022-05-30,SSSO333


In [0]:
Quarterly_Report_df = Query_result_df.replace(99999999,20991231).sort_values(by = ['date_of_suppression']).reset_index(drop = True)
Quarterly_Report_df.dtypes

Out[8]: company_code object
aia_policy_no object
IL_contract_no object
letter_type object
sub_letter_type object
reason_cd object
suppression_date_from int32
suppression_date_to int32
validflag object
user_profile object
date_of_suppression object
Endorser_lanid object
dtype: object

In [0]:
try:
  Quarterly_Report_df['date_of_suppression'] = pd.to_datetime(Quarterly_Report_df['date_of_suppression'], format = '%Y-%m-%d')
  Quarterly_Report_df['suppression_date_from'] = pd.to_datetime(Quarterly_Report_df['suppression_date_from'], format = '%Y%m%d')
  Quarterly_Report_df['suppression_date_to'] = pd.to_datetime(Quarterly_Report_df['suppression_date_to'], format = '%Y%m%d')
  print(Quarterly_Report_df.dtypes)
except:
  if len(Quarterly_Report_df) == 0:
    print('No Record Extracted')
  else:
    print('Error Occured')


company_code object
aia_policy_no object
IL_contract_no object
letter_type object
sub_letter_type object
reason_cd object
suppression_date_from datetime64[ns]
suppression_date_to datetime64[ns]
validflag object
user_profile object
date_of_suppression datetime64[ns]
Endorser_lanid object
dtype: object

In [0]:
Quarterly_Report_df.head()

Out[10]:

,company_code,aia_policy_no,IL_contract_no,letter_type,sub_letter_type,reason_cd,suppression_date_from,suppression_date_to,validflag,user_profile,date_of_suppression,Endorser_lanid
0,2,L546281532,61935825,ZPSFSURR,ZPSFSURR,DPRC,2022-04-01,2022-04-01,1,SSSO256,2022-04-01,SSSO256
1,2,P511394522,00090292,ZCMCTMSR,ZCMCTMSR,DPRC,2022-04-01,2022-04-01,1,SSSO256,2022-04-01,SSSO256
2,2,H238999763,53671799,ZCMCTMSR,ZCMCTMSR,SYSR,2022-03-31,2022-04-01,1,SSSO333,2022-04-01,SSSO333
3,2,H235599438,04240635,ZCMCTMSR,ZCMCTMSR,SYSR,2022-03-31,2022-04-01,1,SSSO333,2022-04-01,SSSO333
4,2,E235599438,56994686,ZCMCTMSR,ZCMCTMSR,SYSR,2022-03-31,2022-04-01,1,SSSO333,2022-04-01,SSSO333


Email Automation

In [0]:
import smtplib
import csv
from string import Template

from email import encoders
from email.mime.base import MIMEBase
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication

In [0]:
subject = "Quarterly Suppression Letter Summary Report ("+str(quarter_start)+" to "+str(quarter_end)+")"
print(subject)

Quarterly Suppression Letter Summary Report (2022-04-01 to 2022-06-30)

In [0]:
body_msg_with = """
Hi Team,

Attached list of """+str(subject)+""". 

Regards
Life Ops, AOS Data Analytics Team
"""

body_msg_without = """
Hi Team,

No record found for """+str(subject)+""". 

Regards
Life Ops, AOS Data Analytics Team
"""

print(body_msg_with,body_msg_without)

Hi Team,

Attached list of Quarterly Suppression Letter Summary Report (2022-04-01 to 2022-06-30). 

Regards
Life Ops, AOS Data Analytics Team
 
Hi Team,

No record found for Quarterly Suppression Letter Summary Report (2022-04-01 to 2022-06-30). 

Regards
Life Ops, AOS Data Analytics Team

In [0]:
sender_email = "SGP.LO-AdvOpsSolns-DataAnalytics@aia.com"
receiver_email = "jimmy.li@aia.com"
cc_email = "jimmy.li@aia.com"

In [0]:
msg = MIMEMultipart()
msg['From'] = sender_email
msg['To'] = receiver_email
msg['Subject'] = subject
msg['cc'] = cc_email
body_with = body_msg_with
body_without = body_msg_without

In [0]:
import io

def export_csv(df):
  with io.StringIO() as buffer:
    df.to_csv(buffer)
    return buffer.getvalue()

In [0]:
Exporter = {'Quarterly Suppression Report.csv': export_csv}

if len(Quarterly_Report_df) == 0:
  msg.attach(MIMEText(body_without,'plain'))
else:
  if len(Quarterly_Report_df) > 0:
    for filename in Exporter:
      attachment1 = MIMEApplication(Exporter[filename](Quarterly_Report_df))
      attachment1['Content-Disposition'] = 'attachment; filename="{}"'.format(filename)
      msg.attach(attachment1)
      
  msg.attach(MIMEText(body_with, 'plain'))
  msg.attach(MIMEText(body_with, 'html'))

print("Email is Ready, about to send out.")

Email is Ready, about to send out.

In [0]:
email_trigger = smtplib.SMTP(host = '10.111.195.29', port=25, timeout = 120)
email_trigger.connect("10.111.195.29",25)

email_trigger.ehlo()
email_trigger.starttls()
email_trigger.ehlo()

email_trigger.send_message(msg)
email_trigger.quit()

print("Whole Process Completed.")

Whole Process Completed.